In [1]:
#importing the Libraies
import numpy as np
import pandas as pd

In [2]:
# Reading the Dataset
dataset = pd.read_csv('insurance_pre.csv')

In [3]:
dataset

,age,sex,bmi,children,smoker,charges
0,19,female,27.900,0,yes,16884.92400
1,18,male,33.770,1,no,1725.55230
2,28,male,33.000,3,no,4449.46200
3,33,male,22.705,0,no,21984.47061
4,32,male,28.880,0,no,3866.85520
...,...,...,...,...,...,...
1333,50,male,30.970,3,no,10600.54830
1334,18,female,31.920,0,no,2205.98080
1335,18,female,36.850,0,no,1629.83350
1336,21,female,25.800,0,no,2007.94500


In [4]:
dataset=pd.get_dummies(dataset,dtype=int,drop_first=True)

In [5]:
dataset

,age,bmi,children,charges,sex_male,smoker_yes
0,19,27.900,0,16884.92400,0,1
1,18,33.770,1,1725.55230,1,0
2,28,33.000,3,4449.46200,1,0
3,33,22.705,0,21984.47061,1,0
4,32,28.880,0,3866.85520,1,0
...,...,...,...,...,...,...
1333,50,30.970,3,10600.54830,1,0
1334,18,31.920,0,2205.98080,0,0
1335,18,36.850,0,1629.83350,0,0
1336,21,25.800,0,2007.94500,0,0


In [6]:
indep=dataset[['age', 'bmi', 'children','sex_male', 'smoker_yes']]
dep=dataset[['charges']]

In [7]:
#split into training set and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(indep, dep, test_size = 0.20, random_state = 0)

In [8]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [10]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
param_grid = {
    'n_estimators': [10, 20, 30, 100],
    'max_depth': [None, 10, 20, 30],
    'max_features': ['sqrt', 'log2'],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}
grid = GridSearchCV(RandomForestRegressor(), param_grid, refit = True, verbose = 3 ,n_jobs=-1)
# fitting the model for grid search
grid.fit(X_train, y_train)

Fitting 5 folds for each of 576 candidates, totalling 2880 fits


C:\Users\rames\anaconda3\envs\NewML\lib\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


GridSearchCV(estimator=RandomForestRegressor(), n_jobs=-1,
             param_grid={'bootstrap': [True, False],
                         'max_depth': [None, 10, 20, 30],
                         'max_features': ['sqrt', 'log2'],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [10, 20, 30, 100]},
             verbose=3)

In [11]:
# Assuming 'grid' is your GridSearchCV object and has been fitted to the data
print("Best parameters:", grid.best_params_)
# Access the mean test score for the best parameters
best_index = grid.best_index_
best_r2_score = grid.cv_results_['mean_test_score'][best_index]
# Printing the best R² score
print("r2:",best_r2_score)

Best parameters: {'bootstrap': True, 'max_depth': 30, 'max_features': 'sqrt', 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 100}
r2: 0.8333880655060348


In [12]:
table=pd.DataFrame.from_dict(grid.cv_results_)

In [13]:
table.sort_values('rank_test_score')

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_bootstrap,param_max_depth,param_max_features,param_min_samples_leaf,param_min_samples_split,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
251,0.368289,0.031080,0.017119,0.003754,True,30,sqrt,4,10,100,"{'bootstrap': True, 'max_depth': 30, 'max_feat...",0.861293,0.835746,0.826800,0.851963,0.791139,0.833388,0.024322,1
239,0.386673,0.185492,0.015504,0.003047,True,30,sqrt,2,10,100,"{'bootstrap': True, 'max_depth': 30, 'max_feat...",0.861845,0.837031,0.825045,0.853652,0.789296,0.833374,0.025485,2
275,0.486607,0.283443,0.021870,0.012507,True,30,log2,2,10,100,"{'bootstrap': True, 'max_depth': 30, 'max_feat...",0.863373,0.837872,0.825919,0.849055,0.787334,0.832711,0.025842,3
243,0.483640,0.361257,0.014013,0.002608,True,30,sqrt,4,2,100,"{'bootstrap': True, 'max_depth': 30, 'max_feat...",0.863360,0.835487,0.824727,0.847418,0.787940,0.831786,0.025406,4
83,0.193043,0.019880,0.013560,0.002560,True,10,sqrt,1,10,100,"{'bootstrap': True, 'max_depth': 10, 'max_feat...",0.865585,0.836075,0.822712,0.848780,0.785725,0.831775,0.027032,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
289,0.168773,0.119553,0.009417,0.007690,False,None,sqrt,1,2,20,"{'bootstrap': False, 'max_depth': None, 'max_f...",0.826658,0.792819,0.761132,0.797566,0.734721,0.782579,0.031695,572
432,0.131232,0.161059,0.003144,0.006287,False,20,sqrt,1,2,10,"{'bootstrap': False, 'max_depth': 20, 'max_fea...",0.815343,0.801493,0.760367,0.785056,0.745068,0.781465,0.025813,573
324,0.209679,0.325768,0.003131,0.006261,False,None,log2,1,2,10,"{'bootstrap': False, 'max_depth': None, 'max_f...",0.814822,0.780114,0.767687,0.791050,0.742036,0.779142,0.024182,574
540,0.144101,0.125976,0.003129,0.006257,False,30,log2,1,2,10,"{'bootstrap': False, 'max_depth': 30, 'max_fea...",0.810884,0.800053,0.743426,0.809915,0.728865,0.778629,0.035196,575


In [14]:
# Assuming 'grid' is your GridSearchCV object that has been fitted to the data
best_model = grid.best_estimator_

In [15]:
import pickle
filename="RF_G.sav"
pickle.dump(best_model,open(filename,'wb'))